In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm
from distill.repsim_util import get_reps

[nltk_data] Downloading package punkt to /home/samira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
chkpt_dir='../tf_ckpts'
task_name='svhn'
task = TASKS[task_name](get_task_params(), data_dir='../data')

In [3]:
hparams=get_model_params(task, 'resnet', 'rsnt_shvn1')
print(hparams)

model config: rsnt_shvn1
{}


In [4]:
config={'exp_name':'test',
    'model_config':'rsnt_svhn1',
    'task_name':'mnist',
    'model_name':'resnet',
    'chkpt_dir':'../tf_ckpts',
    'learning_rate': 0.001
    }

task = TASKS[task_name](get_task_params(), data_dir='../data')
model, _ = get_model(config, task, hparams, 0)


No checkpoint found ../tf_ckpts/svhn/resnet_h-512_rd-2_hdrop-0.2_indrop-0.0_rsnt_svhn1_0.001_test


In [5]:
for x,y in task.train_dataset:
    print(x.shape, y.shape)
    break
model(x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=task.get_loss_fn(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

(64, 32, 32, 3) (64,)
Model: "resnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  896       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
resnet_block (ResnetBlock)   multiple                  18752     
_________________________________________________________________
resnet_block_1 (ResnetBlock) multiple                  18752     
_________________________________________________________________
conv2d_6 (Conv2D)            multiple                  9248      
_________________________________________________________________
global_average_pooling2d (Gl multiple 

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=task.get_loss_fn(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

history = model.fit(task.train_dataset,
                  epochs=30,
                  steps_per_epoch=task.n_train_batches,
                  validation_steps=task.n_valid_batches,
                  validation_data=task.valid_dataset,
                  verbose=2
                  )



Train for 1144 steps, validate for 406 steps
Epoch 1/30
1144/1144 - 10s - loss: 2.0271 - sparse_categorical_accuracy: 0.3022 - val_loss: 1.7871 - val_sparse_categorical_accuracy: 0.3656
Epoch 2/30
1144/1144 - 9s - loss: 1.4214 - sparse_categorical_accuracy: 0.5078 - val_loss: 1.4399 - val_sparse_categorical_accuracy: 0.5017
Epoch 3/30
1144/1144 - 9s - loss: 1.0702 - sparse_categorical_accuracy: 0.6413 - val_loss: 0.9712 - val_sparse_categorical_accuracy: 0.6943
Epoch 4/30
1144/1144 - 9s - loss: 0.7991 - sparse_categorical_accuracy: 0.7423 - val_loss: 0.7962 - val_sparse_categorical_accuracy: 0.7424
Epoch 5/30
1144/1144 - 9s - loss: 0.6339 - sparse_categorical_accuracy: 0.8007 - val_loss: 0.6612 - val_sparse_categorical_accuracy: 0.8038
Epoch 6/30
1144/1144 - 9s - loss: 0.5371 - sparse_categorical_accuracy: 0.8307 - val_loss: 0.5387 - val_sparse_categorical_accuracy: 0.8289
Epoch 7/30
1144/1144 - 9s - loss: 0.4785 - sparse_categorical_accuracy: 0.8500 - val_loss: 0.4657 - val_sparse_cat

In [ ]:
model.evaluate(task.test_dataset, steps=task.n_test_batches)